In [1]:
print("Hello World")

import pandas as pd
import matplotlib as plt
import seaborn as sns
import ipywidgets as ipy

df = pd.read_csv("data/YargaitGH-Ashley-Humid-Solar-Data-May.csv")

df.head(5)

Hello World


,MeasureDate,MeasureTime,SunRadiation1,Temp1,RH1,Temp2,RH2,SunRadiotion2
0,4/30/2023,17:59:00,0.19,17.7,96.5,NaN,NaN,-0.03
1,4/30/2023,18:00:00,0.17,17.7,96.8,21.8,66.5,0.44
2,4/30/2023,18:01:00,0.19,17.7,96.9,21.7,66.5,1.85
3,4/30/2023,18:02:00,0.18,17.7,97.0,21.6,67.6,1.82
4,4/30/2023,18:03:00,0.18,17.7,97.0,21.5,67.2,1.82


In [2]:
# Thermal Load тооцоолол

df = pd.read_csv("data/Gdoc-Weekly-Greenhouse - Jan-1-7.csv")
df.head(5)

,MeasureTime,Fan-1,Fan-2,T-inside,T_fan_input,T-outside,Front_fan_pipe,T-soil-surface,T-soil-deep,Radiation-outside,Radiation-inside,SunRay/100,Temperature difference,Conduction HeatLoss,Solar Power
0,1/1/2023 17:30,0.33,25.7,1.1,1.8,-25.9,0.6,3.2,2.5,2.21,1.75,0.0175,-27.0,-1452.570757,87.5
1,1/1/2023 17:31,0.20,30.7,1.0,1.8,-25.8,0.5,3.3,2.3,2.10,1.75,0.0175,-26.8,-1441.810974,87.5
2,1/1/2023 17:32,-0.20,20.4,1.0,1.8,-25.8,0.5,2.7,2.3,2.00,1.65,0.0165,-26.8,-1441.810974,82.5
3,1/1/2023 17:33,-0.47,27.3,0.9,1.8,-26.1,0.5,2.5,2.3,1.89,1.54,0.0154,-27.0,-1452.570757,77.0
4,1/1/2023 17:34,0.73,25.2,0.8,1.7,-26.1,0.5,2.5,2.4,1.79,1.54,0.0154,-26.9,-1447.190865,77.0
